# IMPORT

In [ ]:
import tempfile
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

# LOAD DATA, TRAIN MODEL

In [ ]:
%%bash

if [ ! -d ./data ]; then
    mkdir ./data
fi

if [ ! -f ./data/trainingandtestdata.zip ]; then
    wget -q -O ./data/trainingandtestdata.zip http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
fi

unzip -n ./data/trainingandtestdata.zip -d ./data

In [ ]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
df_train_file_path = './data/training.1600000.processed.noemoticon.csv'
df_train_name = 'training.1600000.processed.noemoticon'
df_train = pd.read_csv(df_train_file_path,
                      header=None,
                      encoding='ISO-8859-1')

df_test_file_path = './data/testdata.manual.2009.06.14.csv'
df_test_name = 'testdata.manual.2009.06.14'
df_test = pd.read_csv(df_test_file_path,
                     header=None,
                     encoding='ISO-8859-1')
df_train.columns = columns
df_test.columns = columns

In [ ]:
import random
# Remove 'neutral' since it isn't in training dataset
df_test['polarity'] = df_test['polarity'].replace(2, random.choice([0, 4]))
# Make labels monotonically increasing [0,1]
df_test['polarity'] = df_test['polarity'].replace(4, 1)
df_train['polarity'] = df_train['polarity'].replace(4, 1)

In [ ]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df=100,
                                                        ngram_range=(1,2),
                                                        stop_words='english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(df_train.text, df_train.polarity)

In [ ]:
x_test, y_test = df_test.text, df_test.polarity
print(classification_report(y_test, sentiment_lr.predict(x_test)))

In [ ]:
sentiment_lr.predict([x_test[0]])

In [ ]:
sentiment_lr.predict(['good', 'bad'])

# UNBOX

In [ ]:
import unboxapi
client = unboxapi.UnboxClient("API_KEY_HERE")

## Create function

In [ ]:
def predict_function(model, text_list):
    return model.predict_proba(text_list)

In [ ]:
texts = ['some new text, sweet noodles', 'happy time', 'sad day']

predict_function(sentiment_lr, texts)

# Package (function, model) & Upload to Unbox Server

### Upload dataset from dataframe

In [ ]:
dataset = client.add_dataframe(
    df=df_test,
    class_names=['negative', 'positive'],
    label_column_name='polarity',
    text_column_name='text',
    name=df_test_name,
    description='this is my sentiment test dataset'
)
dataset.to_dict()

### Upload model

In [ ]:
from unboxapi.model import ModelType

model = client.add_model(
    function=predict_function, 
    model=sentiment_lr,
    model_type=ModelType.sklearn,
    class_names=['negative', 'positive'],
    name='05.15.2021.sentiment_analyzer',
    description='this is my sklearn sentiment model'
)
model.to_dict()

### Upload dataset from csv

In [ ]:
print('\nUploading dataset (from file)...')
with tempfile.TemporaryDirectory() as tmpdir:
    df_train.to_csv(f'{tmpdir}/dataset.csv')
    dataset = client.add_dataset(
        file_path=f'{tmpdir}/dataset.csv', 
        class_names=['negative', 'positive'],
        label_column_name='polarity',
        text_column_name='text'
        name=df_train_name,
        description='this is my sentiment train dataset',
    )
dataset.to_dict()